# Solving SAT With Grover
In this notebook, we aim to solve satisfyability formulae using grover search.
For this, we first construct marking oracles for the basic gates of AND, OR and NOT.
Then, we transform them into the required phase-oracle-form.

The main resources used are this [Microsoft Kata](https://github.com/microsoft/QuantumKatas/blob/ec925ccfcb599a6bb29c8a39035d0b149f447f9a/SolveSATWithGrover/Workbook_SolveSATWithGrover.ipynb) and this [Qiskit Textbook Chapter](https://qiskit.org/textbook/ch-applications/satisfiability-grover.html).

## Marking Oracles
### And

In [ ]:
from qiskit.quantum_info.operators import Operator
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister, ClassicalRegister
import matplotlib.pyplot as plt

## Define Basic Matrices
I = Operator([[1,0],[0,1]])
X = Operator([[0,1],[1,0]])

def sized_identity(nqubits):
    """Creates an identity operator of given size"""
    unitary = I
    for i in range(nqubits-1):
        unitary = unitary.tensor(I)
    return unitary

def marking_and_oracle(nqubits):
    """Returns a matrix representation of the and oracle"""
    unitary = sized_identity(nqubits)
    
    # Apply the x gate to the output register
    unitary.data[-1][-1] = 0
    unitary.data[-2][-2] = 0
    unitary.data[-1][-2] = 1
    unitary.data[-2][-1] = 1
    return unitary 

### Or

In [ ]:
def sized_x(nqubits):
    """Creates a not operator of given size"""
    unitary = X
    for i in range(nqubits-1):
        unitary = unitary.tensor(X)
    return unitary

def marking_or_oracle(nqubits):
    """Returns a matrix representation of the or oracle"""
    unitary = sized_x(nqubits).dot(marking_and_oracle(nqubits))
    unitary = unitary.dot(sized_x(nqubits-1).tensor(I))
    return unitary

While we could go on further and create _XOR_ and _AlternatingBits_ oracles but we don't need them for now so let's not :)

## SAT Marking Oracles
Using our marking gate-oracles, we can create oracles for evaluating SAT clauses and finally complete functions.

### Single Clause Oracle
A Clause is a disjunction of variables (qubits) that are potentially negated.
The clause `x0 || !x1` may be represented by the input `[(0, true), (1, false)]`.

In [ ]:
def marking_clause_oracle(qubits: [int], target: int, clause: [(int,bool)]):
    nqubits = len(qubits) + 1
    qc = QuantumCircuit(nqubits)
    # Flip input qubits
    for index, positive in clause:
        if not positive:
            qc.x(index)
            
    # Get Clause Qubits
    clause_qubits = [i for i, _ in clause]
    
    # Create an Oracle for n qubits
    nqubits_or = len(clause_qubits) + 1  # +1 for target qubit
    or_oracle = marking_or_oracle(nqubits_or)
    or_qc = QuantumCircuit(nqubits_or)
    or_qc.unitary(or_oracle, range(nqubits_or), "or_oracle")
    # compose list maps the qubits linearly (0,1,2,3...) on the given target list
    clause_qubits.append(target)
    qc = qc.compose(or_qc, clause_qubits)
    
    # Inverse the initial Flips
    for index, positive in clause:
        if not positive:
            qc.x(index)
    return qc

In [ ]:
# Debug Clause Oracle
clause_circuit = marking_clause_oracle(range(5), 5, [(1, True),(3, False),(4, True)])
clause_circuit.draw(output="mpl")

### k-SAT Oracle
Using our single clause oracle, we can take their conjunction and create a marking oracle for the entire expression.

In [ ]:
def marking_ksat_oracle(qubits: [int], target: int, clauses: [[(int, bool)]]):
    auxiliary_qc = QuantumCircuit(len(clauses)) # Each clause output needs to be stored in aux
    qreg_in = QuantumRegister(size=len(qubits), name="q_in")
    qreg_an = AncillaRegister(size=len(clauses), name="q_an")
    qreg_target = QuantumRegister(size=1, name="q_target")
    
    qc = QuantumCircuit(qreg_in, qreg_an, qreg_target)
    
    # Compose individual clauses
    for idx, clause in enumerate(clauses):
        aux_idx = len(qubits) + idx
        one_clause_qc = marking_clause_oracle(qubits, len(qubits), clause)
        qbit_map = list(qubits)
        qbit_map.append(aux_idx)
        qc = qc.compose(one_clause_qc, qbit_map)
    
    # Store the conjugate transpose (inverse) for later qubit cleanup
    inverse_qc = qc.inverse()
    
    # And the auxiliary registers
    nqubits_and = len(clauses) + 1
    and_oracle = marking_and_oracle(nqubits_and)
    and_qc = QuantumCircuit(nqubits_and)
    and_qc.unitary(and_oracle, range(nqubits_and), "and_oracle")
    
    # Compose the and oracle
    and_qc_map = [idx+len(qubits) for idx in range(len(clauses)+1)]
    ksat_qc = qc.compose(and_qc, and_qc_map)
    # Apply the inversion to the circuit
    return ksat_qc.compose(inverse_qc)

def ksat_for_problem(problem):
    nqubits_aux = len(problem) # one for each clause
    nqubit_in = max([i for c in problem for i, _ in c]) + 1
    problem_qc = marking_ksat_oracle(range(nqubit_in), nqubit_in+nqubits_aux+1, problem)
    return problem_qc
    

**DONE** We can now formulate a SAT problem and create a marking oracle for it!

In [ ]:
# f(x) = (x1 + !x3 + x4) * (x0 + !x2)
problem = [[(1, True),(3, False),(4, True)],
          [(0, True),(2, False)]]
problem_circuit = ksat_for_problem(problem)
problem_circuit.draw(output="mpl", justify="left")


## Converting the oracles into phase oracles for Grover

In [ ]:
def phase_oracle(problem_oracle, target_idx):
    qc_conv = QuantumCircuit(1, name="Mark2Phase")
    qc_conv.x(0)
    qc_conv.h(0)
    # Prepend the phase transformation
    problem_oracle = problem_oracle.compose(qc_conv, qubits=[target_idx], front=True)
    # Append the phase transformation 
    problem_oracle = problem_oracle.compose(qc_conv.inverse(), qubits=[target_idx])
    
    return problem_oracle

phase_problem_circuit = phase_oracle(problem_circuit, -1)
phase_problem_circuit.draw(output="mpl")

# Plugging Oracles into Grover
As we can now create phase oracles, we can model a complete grover loop.

1. Make oracle: ksat_for_problem(...) & phase_oracle(...)
2. Add classical registers for output measurements
3. Create uniform superposition over q_in (an & target stay |0>)
4. Grover Loop (k-times? correctness-check?)
    1. Append Phase_Oracle over all q registers
    2. Append Grover Diffuser over all q registers
5. Measure q_in into classical registers and check solution (histogram)

In [ ]:
def create_diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_{Diffuser}$"
    return U_s

In [ ]:
# Input:
# f(x) = (x0 + !x1) * (x2) * (x1) {FTT, TTT}
problem = [[(0, True), (1, False)], [(2, True)], [(1, True)]]

marking_oracle = ksat_for_problem(problem)
phase_problem_qc = phase_oracle(marking_oracle, -1)
oracle_gate = phase_problem_qc.to_gate(label="SAT_Oracle")
num_var = phase_problem_qc.num_qubits-phase_problem_qc.num_ancillas

diffuser = create_diffuser(num_var-1)

# Construct Main Quantum Circuit
main_qc = phase_problem_qc.copy_empty_like()
creg = ClassicalRegister(num_var, name="c")
main_qc.add_register(creg)

# main_qc.draw(output="mpl", justify="left")

(3.) Create uniform superposition over input qubits

In [ ]:
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc

main_qc = initialize_s(main_qc, range(num_var-1))
# main_qc.h(-1)

# main_qc.draw(output="mpl", justify="left")

(4.) Grover Loop

In [ ]:
# TODO find k
k=1

# Add the oracle-diffusor step k times
for i in range(k):
    main_qc.append(oracle_gate, range(main_qc.num_qubits))
    register_map = list(range(num_var-1))
    # register_map.append(-1) # target
    main_qc = main_qc.compose(diffuser, register_map)
    
# Measure Inputs and Target
for i, c in enumerate(creg):
    if i < len(creg)-1:
        main_qc.measure(i, c) # measure inputs
#     else:
#         main_qc.measure(-1, c) # measure target, if 0 the results are trash

# Reapply Marking Oracle for correctness check
qreg_out = QuantumRegister(size=1, name="q_out")
main_qc.add_register(qreg_out)
marking_out_map = list(range(num_var+main_qc.num_ancillas-1))
marking_out_map.append(-1)
main_qc = main_qc.compose(marking_oracle, marking_out_map)
main_qc.measure(-1, -1) # measure q_out into last classical register

main_qc.draw(output="mpl", justify="left")

The Quantum Circuit is completed! Simulate it!

In [ ]:
from qiskit import Aer, transpile
from qiskit.visualization import plot_histogram

qasm_sim = Aer.get_backend('qasm_simulator')
transpiled_grover_circuit = transpile(main_qc, qasm_sim)
results = qasm_sim.run(transpiled_grover_circuit).result()
counts = results.get_counts()
histogram = plot_histogram(counts)
histogram # For display purposes in jupyter

In [ ]:
# DMACS
problem_dmacs = "c problem dmacs\np cnf 3 2\n1 -2 0\n3 0"

from qiskit.circuit.library import PhaseOracle
import numpy as np
np.set_printoptions(threshold=np.inf)

import qiskit.quantum_info as qi
op = qi.Operator(phase_problem_qc)
print(op.data.real)